In [10]:
# Import Block
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Collaborative Filtering

## Singular Value Decomposition (SVD)


In [11]:
# Load data
df_interactions_train = pd.read_csv("data/interactions_train_processed.csv")

# Load data into Surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_interactions_train[['user_id', 'recipe_id', 'rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train SVD model
svd = SVD()
svd.fit(trainset)

# Evaluate the model
predictions = svd.test(testset)
rmse(predictions)

RMSE: 0.9209


0.9209006108026337

# Content Filtering

## Cosine Similarity


In [ ]:
# TF-IDF on ingredients
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_interactions_train['ingredient_names'])

# Function to get recommendations
def content_based_recommendations(recipe_name, top_n=10):
    idx = df_interactions_train[df_interactions_train['recipe_name'] == recipe_name].index[0]
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]
    recipe_indices = [i[0] for i in sim_scores]
    return df_interactions_train.iloc[recipe_indices][['recipe_name', 'ingredient_names']]

# Test the function
content_based_recommendations('greek stuffed meatloaf') #Chocolate Cake')


,recipe_name,ingredient_names
456802,greek stuffed meatloaf,"['lamb', 'hamburger', 'egg', 'milk', 'quick oa..."
622519,lamburgers with tzatziki sauce,"['ground lamb', 'feta cheese', 'greek olive', ..."
137134,zucchini moussaka,"['olive oil', 'zucchini', 'onion', 'ground lam..."
208539,zucchini moussaka,"['olive oil', 'zucchini', 'onion', 'ground lam..."
142241,moussaka aunt dana style,"['zucchini', 'olive oil', 'ground lamb', 'onio..."
575670,moussaka aunt dana style,"['zucchini', 'olive oil', 'ground lamb', 'onio..."
82465,lamb spinach feta burgers,"['ground lamb', 'frozen chopped spinach', 'oni..."
141206,lamb spinach feta burgers,"['ground lamb', 'frozen chopped spinach', 'oni..."
387126,lamb spinach feta burgers,"['ground lamb', 'frozen chopped spinach', 'oni..."
459904,lamb spinach feta burgers,"['ground lamb', 'frozen chopped spinach', 'oni..."
